In [22]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [24]:
#obtain list of dates
directory = "Boston Celtics"
list_dates = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        list_dates.append(filename)

In [25]:
# create dictionary 
final_data = {}

In [29]:
# reads in each csv
for i in list_dates:
    data = pd.read_csv(f"Boston Celtics/{i}", index_col=0)
    
    #gathers data and adds to dict
    date = data["Date"].tolist()[0]
    data[date] = data["Score"]
    final_data[date] = data[date]

In [30]:
# convert dict to dataframe
data_df = pd.DataFrame(final_data)

In [31]:
tester = pd.DataFrame.transpose(data_df)
columns = tester.columns.tolist()

In [32]:
#create dict
final_result = {}

In [33]:
tester

,Aaron Nesmith,Al Horford,Brodric Thomas,Daniel Theis,Derrick White,Grant Williams,Jaylen Brown,Jayson Tatum,Juwan Morgan,Luke Kornet,Malik Fitts,Marcus Smart,Matt Ryan,Nik Stauskas,Payton Pritchard,Robert Williams III,Sam Hauser
06/16/2022,0.00,46.00,NaN,NaN,5.75,7.25,49.25,33.25,0.00,0.75,NaN,33.00,NaN,0.00,2.25,30.75,0.00
06/02/2022,0.00,42.00,NaN,5.50,28.25,8.25,45.25,39.25,0.00,NaN,0.00,36.25,NaN,0.00,19.50,24.50,0.00
04/10/2022,8.75,25.00,12.0,26.75,18.00,16.25,27.25,49.25,NaN,19.25,14.25,17.75,5.5,NaN,26.00,NaN,9.50
06/10/2022,NaN,24.00,NaN,NaN,20.75,4.75,33.50,52.75,NaN,NaN,NaN,39.00,NaN,NaN,1.75,34.00,NaN
06/05/2022,11.50,14.50,NaN,9.25,25.50,11.25,29.50,43.00,NaN,3.50,3.50,11.50,NaN,8.75,13.50,10.00,5.00
06/13/2022,4.25,21.75,NaN,NaN,8.75,11.25,34.75,46.00,1.25,4.75,NaN,26.25,NaN,0.00,0.75,23.00,1.50
04/17/2022,NaN,44.25,NaN,13.00,22.25,13.00,45.25,53.00,NaN,NaN,NaN,42.25,NaN,NaN,4.75,NaN,NaN
05/25/2022,0.00,38.25,NaN,NaN,27.25,12.50,36.00,49.00,0.00,0.00,0.00,17.25,NaN,0.00,1.25,26.50,NaN
05/19/2022,1.25,21.75,NaN,4.75,NaN,27.00,40.00,43.25,2.25,4.50,NaN,63.25,NaN,7.75,18.50,17.50,NaN
05/27/2022,0.00,20.75,NaN,NaN,43.25,6.50,37.50,51.75,NaN,NaN,NaN,25.00,NaN,NaN,0.00,18.75,NaN


In [44]:
add = {}

#iterate through index
for column in columns:
    print(column)
    
# univariate data prep LSTM
    from numpy import array

    # split sequence
    def split_sequence(sequence, n_steps):
        X, y = list(), list()
        for i in range(len(sequence)):
            # find end of pattern 
            end_ix = i + n_steps
            #check if beyond sequence to break
            if end_ix > len(sequence)-1:
                break
            #gather input and output parts of the pattern
            seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)
    # sequence 
    rent_rate = list(tester[column].dropna())
    # number of steps
    n_steps = 3
    if len(rent_rate) < 5:
        continue
    X, y = split_sequence(rent_rate, n_steps)
    # summarize the data
    for i in range(len(X)):
        print(X[i], y[i])

    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = 1
    X = X.reshape((X.shape[0], X.shape[1], n_features))

    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # fit model
    model.fit(X, y, epochs=200, verbose=0)

    # demonstrate prediction
    x_input = array(rent_rate[-3:])
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    add[column] = yhat.tolist()[0][0]

Aaron Nesmith
[0.   0.   8.75] 11.5
[ 0.    8.75 11.5 ] 4.25
[ 8.75 11.5   4.25] 0.0
[11.5   4.25  0.  ] 1.25
[4.25 0.   1.25] 0.0
[0.   1.25 0.  ] 10.25
[ 1.25  0.   10.25] 0.0
[ 0.   10.25  0.  ] 1.25
[10.25  0.    1.25] 8.75
[0.   1.25 8.75] 1.25
[1.25 8.75 1.25] 4.75
[8.75 1.25 4.75] 2.75
[1.25 4.75 2.75] 1.25
Al Horford
[46. 42. 25.] 24.0
[42. 25. 24.] 14.5
[25.  24.  14.5] 21.75
[24.   14.5  21.75] 44.25
[14.5  21.75 44.25] 38.25
[21.75 44.25 38.25] 21.75
[44.25 38.25 21.75] 20.75
[38.25 21.75 20.75] 34.25
[21.75 20.75 34.25] 48.0
[20.75 34.25 48.  ] 48.5
[34.25 48.   48.5 ] 34.5
[48.  48.5 34.5] 55.5
[48.5 34.5 55.5] 28.5
[34.5 55.5 28.5] 39.25
[55.5  28.5  39.25] 33.5
[28.5  39.25 33.5 ] 29.5
[39.25 33.5  29.5 ] 36.0
[33.5 29.5 36. ] 34.5
[29.5 36.  34.5] 23.0
[36.  34.5 23. ] 34.0
[34.5 23.  34. ] 15.25
Brodric Thomas
Daniel Theis
[ 5.5  26.75  9.25] 13.0
[26.75  9.25 13.  ] 4.75
[ 9.25 13.    4.75] 7.75
[13.    4.75  7.75] 5.25
[4.75 7.75 5.25] 2.25
[7.75 5.25 2.25] 14.75
[ 5

In [45]:
add

{'Aaron Nesmith': 1.4071203470230103,
 'Al Horford': 16.850967407226562,
 'Daniel Theis': 15.930194854736328,
 'Derrick White': 13.809983253479004,
 'Grant Williams': 25.694034576416016,
 'Jaylen Brown': 40.865169525146484,
 'Jayson Tatum': 50.12757873535156,
 'Juwan Morgan': 0.7012506127357483,
 'Luke Kornet': 0.18349812924861908,
 'Malik Fitts': -0.8474617600440979,
 'Marcus Smart': 28.674304962158203,
 'Nik Stauskas': 0.48079511523246765,
 'Payton Pritchard': 4.313274383544922,
 'Robert Williams III': 25.870563507080078,
 'Sam Hauser': 0.31054550409317017}

In [46]:
# convert dict to dataframe
Data = pd.DataFrame.from_dict(add, orient='index')

In [47]:
Data

,0
Aaron Nesmith,1.407120
Al Horford,16.850967
Daniel Theis,15.930195
Derrick White,13.809983
Grant Williams,25.694035
Jaylen Brown,40.865170
Jayson Tatum,50.127579
Juwan Morgan,0.701251
Luke Kornet,0.183498
Malik Fitts,-0.847462


In [48]:
# convert dataframe to csv
mypath = "Final_Score/LSTM"
if not os.path.isdir(mypath):
    os.makedirs(mypath)
Data.to_csv(f"{mypath}/BostonCeltics.csv", index=True)